# RSA implementation

In [2]:
from math import sqrt
import random
from random import randint as rand
import time

In [3]:
#function to calculate the greatest common divisor
def gcd(a, b):
    if b == 0:
        return a
    else:
        return gcd(b, a % b)

In [4]:
#function to calculate the mod
def mod_inverse(a, m):
    for x in range(1, m):
        if (a * x) % m == 1:
            return x
    return -1

In [5]:
#function to check if its a prime number
def isprime(n):
    if n < 2:
        return False
    elif n == 2:
        return True
    else:
        for i in range(2, int(sqrt(n)) + 1, 2):
            if n % i == 0:
                return False
    return True

In [6]:
#function to calculate the public and private key
def generate_keypair(p, q, keysize):
    nMin = 1 << (keysize - 1)
    nMax = (1 << keysize) - 1
    primes = [2]

    start = 1 << (keysize // 2 - 1)
    stop = 1 << (keysize // 2 + 1)

    if start >= stop:
        return []

    for i in range(3, stop + 1, 2):
        for p in primes:
            if i % p == 0:
                break
        else:
            primes.append(i)
    while (primes and primes[0] < start):
        del primes[0]
        
    while primes:
        p = random.choice(primes)
        primes.remove(p)
        q_values = [q for q in primes if nMin <= p * q <= nMax]
        if q_values:
            q = random.choice(q_values)
            break
    print(p, q)
    n = p * q
    phi = (p - 1) * (q - 1)
    e = random.randrange(1, phi)
    g = gcd(e, phi)

    while True:
        e = random.randrange(1, phi)
        g = gcd(e, phi)
        d = mod_inverse(e, phi)
        if g == 1 and e != d:
            break
    return ((e, n), (d, n))

In [7]:
#function to print present time
from datetime import datetime

def print_time():
    now = datetime.now()
    dt_string = now.strftime("%d/%m/%Y %H:%M:%S")
    print("Time now: ", dt_string)

### Encryption

In [8]:
#function to encrypt the text
def encrypt(msg_plaintext, package):
    e, n = package
    msg_ciphertext = [pow(ord(c), e, n) for c in msg_plaintext]
    return msg_ciphertext

### Decryption

In [9]:
#function to decrypt the data
def decrypt(msg_ciphertext, package):
    d, n = package
    msg_plaintext = [chr(pow(c, d, n)) for c in msg_ciphertext]
    return (''.join(msg_plaintext))

### Input

In [28]:
#generating random numbers, later to be checked if prime
p = rand(1, 1000)
q = rand(1, 1000)

bit_length = int(input("Enter bit_length: "))
public, private = generate_keypair(p, q, 2**bit_length)  # 8 is the keysize (bit-length) value.
print("Public Key: ", public)
print("Private Key: ", private)
print("\n")
print_time()

msg = input("Write text to be encrypted: ")

encrypted_msg = encrypt(msg, public)
print("Encrypted text: ",''.join(map(lambda x: str(x), encrypted_msg)))

decrypted_msg = decrypt(encrypted_msg, private)
print("Decrypted text: ",decrypted_msg)

Enter bit_length: 4
439 139
Public Key:  (25685, 61021)
Private Key:  (5309, 61021)


Time now:  28/09/2021 22:58:39
Write text to be encrypted: He watched as the young man tried to impress everyone in the room with his intelligence. There was no doubt that he was smart. The fact that he was more intelligent than anyone else in the room could have been easily deduced, but nobody was really paying any attention due to the fact that it was also obvious that the young man only cared about his intelligence.
Encrypted text:  290774612830791186798522581935047060793461283232730791852217112307915819360793461283079160525545600833346237863079137452852233462307915819341341176474612832327307915819325545307911764737452897641341461281711217112307914612860512461284134160525545334624612830791176473346230791581936079346128307914134125545255453745230791186791764758193607933079160793176471711230791176473346258193461284289742897176473786461283346250470461282227830791223166079346128413414612830791186798522

In [29]:
print("Response time for encryption is: ")
%timeit encrypt(msg, public)

Response time for encryption is: 
431 µs ± 9.62 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [30]:
print("Response time for decryption is: ")
%timeit decrypt(encrypted_msg, private)

Response time for decryption is: 
471 µs ± 8.88 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
